In [16]:
import os
import numpy as np

from pathlib import Path
from qibolab import create_platform, ExecutionParameters, AcquisitionType, AveragingMode
from qibolab.pulses import PulseSequence
from qibolab.sweeper import Sweeper, Parameter, SweeperType

os.environ["QIBOLAB_PLATFORMS"] = str(Path.cwd() / "platforms")

In [11]:
platform = create_platform("myplatform")
platform

[Qibo 0.2.4|INFO|2024-02-05 16:24:38]: Loading platform myplatform


Platform(name='myplatform', qubits={0: Qubit(name=0, bare_resonator_frequency=0, readout_frequency=7000000000, drive_frequency=4500000000, anharmonicity=0, sweetspot=0.0, asymmetry=0.0, crosstalk_matrix={}, Ec=0.0, Ej=0.0, g=0.0, assignment_fidelity=0.0, readout_fidelity=0.0, effective_temperature=0.0, peak_voltage=0, pi_pulse_amplitude=0, T1=0, T2=0, T2_spin_echo=0, state0_voltage=0, state1_voltage=0, mean_gnd_states=[0, 0], mean_exc_states=[0, 0], threshold=None, iq_angle=0.0, mixer_drive_g=0.0, mixer_drive_phi=0.0, mixer_readout_g=0.0, mixer_readout_phi=0.0, readout=Channel(name='readout', port=DummyPort(name='o1', offset=0.0, lo_frequency=0, lo_power=0, gain=0, attenuation=0, power_range=0, filters=None), local_oscillator=None, max_offset=None), feedback=Channel(name='feedback', port=DummyPort(name='i1', offset=0.0, lo_frequency=0, lo_power=0, gain=0, attenuation=0, power_range=0, filters=None), local_oscillator=None, max_offset=None), twpa=None, drive=Channel(name='drive', port=Du

In [12]:
platform.qubits[0]

Qubit(name=0, bare_resonator_frequency=0, readout_frequency=7000000000, drive_frequency=4500000000, anharmonicity=0, sweetspot=0.0, asymmetry=0.0, crosstalk_matrix={}, Ec=0.0, Ej=0.0, g=0.0, assignment_fidelity=0.0, readout_fidelity=0.0, effective_temperature=0.0, peak_voltage=0, pi_pulse_amplitude=0, T1=0, T2=0, T2_spin_echo=0, state0_voltage=0, state1_voltage=0, mean_gnd_states=[0, 0], mean_exc_states=[0, 0], threshold=None, iq_angle=0.0, mixer_drive_g=0.0, mixer_drive_phi=0.0, mixer_readout_g=0.0, mixer_readout_phi=0.0, readout=Channel(name='readout', port=DummyPort(name='o1', offset=0.0, lo_frequency=0, lo_power=0, gain=0, attenuation=0, power_range=0, filters=None), local_oscillator=None, max_offset=None), feedback=Channel(name='feedback', port=DummyPort(name='i1', offset=0.0, lo_frequency=0, lo_power=0, gain=0, attenuation=0, power_range=0, filters=None), local_oscillator=None, max_offset=None), twpa=None, drive=Channel(name='drive', port=DummyPort(name='o2', offset=0.0, lo_frequ

## Execute

In [13]:
sequence = PulseSequence()
ro_pulse = platform.create_MZ_pulse(qubit=0, start=0)
sequence.add(ro_pulse)

options = ExecutionParameters(
    nshots=1000,
    relaxation_time=100000,
    acquisition_type=AcquisitionType.DISCRIMINATION,
    averaging_mode=AveragingMode.SINGLESHOT

)
results = platform.execute_pulse_sequence(sequence, options)


for key, value in results.items():
    print(key, value)


[Qibo 0.2.4|INFO|2024-02-05 16:24:41]: Minimal execution time (sequence): 0.101


0 <qibolab.result.SampleResults object at 0x747d81a0bf90>
ReadoutPulse(0, 1000, 0.005, 7_000_000_000, 0, Rectangular(), readout, 0) <qibolab.result.SampleResults object at 0x747d81a0bf90>


In [14]:
print(results[ro_pulse.serial].samples.shape)

(1000,)


## Sweep

In [21]:
sequence = PulseSequence()
ro_pulse = platform.create_MZ_pulse(qubit=0, start=0)
sequence.add(ro_pulse)

sweeper = Sweeper(
    parameter=Parameter.frequency,
    values=np.arange(-2e8, +2e8, 1e6),
    pulses=[ro_pulse],
    type=SweeperType.OFFSET,
)

options = ExecutionParameters(
    nshots=1000,
    relaxation_time=1000,
    acquisition_type=AcquisitionType.INTEGRATION,
    averaging_mode=AveragingMode.CYCLIC
)
results = platform.sweep(sequence, options, sweeper)

results[ro_pulse.serial].voltage_i.shape

[Qibo 0.2.4|INFO|2024-02-05 16:26:57]: Minimal execution time (sweep): 0.8


(400,)

## Sequence unrolling

In [25]:
platform = create_platform("myplatform")

nsequences = 20
sequences = []
for _ in range(nsequences):
    sequence = PulseSequence()
    sequence.add(platform.create_MZ_pulse(qubit=0, start=sequence.finish))
    sequences.append(sequence)

options = ExecutionParameters(
    nshots=1000,
    relaxation_time=100000,
    acquisition_type=AcquisitionType.DISCRIMINATION,
    averaging_mode=AveragingMode.SINGLESHOT,
)
results = platform.execute_pulse_sequences(sequences, options)

results[ro_pulse.serial]

[Qibo 0.2.4|INFO|2024-02-05 16:30:37]: Loading platform myplatform
[Qibo 0.2.4|INFO|2024-02-05 16:30:37]: Minimal execution time (unrolling): 2.02


In [26]:
len(results[ro_pulse.serial])

20

In [27]:
results[ro_pulse.serial][0].samples.shape

(1000,)